# Forest Observatory Webinar API demo

This notebook walks through the central `cfo` Python API functions and demonstrates some potential applications. This includes searching for data, downloading it locally, rendering map tiles & clipping data using other spatial datasets.

This notebook is running in the cfo `conda` environment. We created this simple environment to include a series of useful geospatial/data science packages we use on a regular basis (`gdal`, `otb`, `geopandas`, `sklearn`, etc.). If you have conda [installed](https://docs.conda.io/projects/conda/en/latest/user-guide/install/), you can run the following to run this notebook:

```bash
git clone https://github.com/forestobservatory/cfo-api.git
cd cfo-api
conda env update
conda activate cfo
jupyter notebook
```

### What we'll cover

The `cfo` package is pretty simple. We designed it to make it easy for users to access and use CFO data. This notebook will address the following topics:

- Searching for data
- Downloading data
- Loading web maps

You'll need a CFO account to run everything in this notebook. Sign up at [forestobservatory.com](https://forestobservatory.com).

### A note on cloud data storage

This notebook review how to download the data using Salo Science's API. But if you want to skip the middle man (erm, the middle alien) you can find the data on our publicly-accessible Google Cloud Storage Bucket ([link here](https://console.cloud.google.com/storage/browser/cfo-public)). If you use `gsutil`, you can access the data via the `gs://cfo-public` bucket)

In [ ]:
# !!!!!!!!!!!!!!!!!
# change this to a directory on your local machine to determine where downloads will go
output_directory = '/home/cba/cfo'

In [ ]:
# package prep
import os
import cfo
import gdal
import ipyleaflet
import rasterio
import numpy as np
from matplotlib import pyplot as plt
from ipyleaflet import Map, WMSLayer, LayersControl, basemaps

# set plotting style
%matplotlib notebook

# create the output directory if it doesn't exist
if not os.path.exists(output_directory):
    os.mkdir(output_directory)

# Searching for data

Forest Observatory data are organized by `asset_id`, and you'll need to acquire these IDs before you can do anything else. We built the `search` function to facilitate this. It helps to understand the asset ID format:

```python
{geography}-{category}-{metric}-{year}-{timeOfYear}-{resolution}
```

We'll explore these in more detail below.

When you run a `search` command it returns a `list` of asset IDs

In [ ]:
# first, we'll create an api class instance
forest = cfo.api()

In [ ]:
# search for all California-wide vegetation datasets
ca = forest.search(geography='California', category='Vegetation')

# list the results
ca.sort()
for asset in ca:
    print(asset)

In [ ]:
# you can search the daily weather datasets we produce, which are deleted from the api after two weeks

# list the wind speed assets
wind = forest.search(metric='WindSpeed')
wind.sort()
for asset in wind:
    print(asset)

In [ ]:
# all the CFO data on the website have been clipped by County, Watershed and Municipality. 
# you can search those, too. Try it using wildcards

# we'll search an ambiguous area
sc = forest.search(geography='santacruz*')
sc.sort()
for asset in sc:
    print(asset)

### Cool! But, what if I don't already know what to enter into those fancy category="blah blah blah" fields?

We've built some convenience functions to help you out! The search function accepts keywords that match the `asset_id` formatting above. So, `forest.search(geography=str, category=str, metric=str, year=int, timeOfYear=str, resolution=int)`.

The convenience functions (under `forest.list_*` can guide some suitable inputs for each.

In [ ]:
# list the categories of data we generate
forest.list_categories()

In [ ]:
# list the metrics in the Vegetation category
forest.list_metrics('Vegetation')

In [ ]:
# and the weather metrics
forest.list_metrics('Weather')

In [ ]:
# list the different types of geography we slice data by
forest.list_geography_types()

In [ ]:
# list the first few county names
forest.list_geographies('County')[:5]

In [ ]:
# which states do we cover?
forest.list_geographies('State')

:sunglasses:

In [ ]:
# wanna be a rebel and just get every dang asset we've published?
thewholeshebang = forest.search()
print(f"Number of CFO assets: {len(thewholeshebang)}")

:astonished:

In [ ]:
# for more help, run
forest.search?

### Gimme the highest resolution data you have!

Thanks for asking nicely. We're still in the process of publishing our 3m datasets, and we appreciate your patience as we do so. Here's how you would search for the statewide datasets at 10m and 3m resolution.

In [ ]:
ca3 = forest.search(geography='california', resolution=3)
ca10 = forest.search(geography='california', resolution=10)
print(ca3)

# Downloading data

Though `search` returns a list of asset IDs, the `download` function requires string inputs. This is to reduce the risk of downloading a huge list of files by accident.

In [ ]:
# grab the most recent tree height data from grass valley
gv = forest.search(geography="GrassValley*", metric='CanopyHeight', year=2020)
print(f"Number of files found: {len(gv)}")

# iterate over the list to pass the asset id as a string
for asset in gv:
    filename = os.path.join(output_directory, asset + '.tif')
    forest.download(asset, filename)

In [ ]:
# to get all the veg metrics, just iterate over the list
gv_time_series = forest.search(geography="GrassValley*", category="Vegetation")
for asset in gv_time_series:
    filename = os.path.join(output_directory, asset + '.tif')
    forest.download(asset, filename)

### What's happening under the hood?

Great segue. Thanks for asking. The `download` function calls another routine, `fetch()` to get a URL to where the file is stored on our Google Cloud Storage Bucket. The fetch function is used to retrieve a series of different asset-specific URLs.

Downloads use a "signed url", which means you can just directly download the file without needing to authenticate with Google first. We then make a `GET` request using this url to stream the file to disk.

In [ ]:
# print out the long and convoluted signed url
for asset in gv:
    print(forest.fetch(asset, dl=True))

In [ ]:
# alternatively you can just get the direct bucket uri
for asset in gv:
    print(forest.fetch(asset, bucket=True))

In [ ]:
# set multiple flags to return a dictionary of urls
for asset in gv:
    urls = forest.fetch(asset, dl=True, bucket=True, wms=True, gdal=True)
    for key in list(urls.keys()):
        print(f"{key}: {urls[key]}")
        print("")

# Custom data clipping

We clipped all of the 2020 10-meter Vegetation data by County, Municipality & Watershed based on feedback from the community that these were the most relevant areas and datasets of interest. You're welcome.

But we imagine many people are going to be interested in analyzing custom areas of interest. How do you download data from an AOI vector file?

The best way to do this with the API is through `gdal`, referencing the statewide datasets.

`gdalwarp` has a really nice driver for handling [cloud datasets](https://gdal.org/user/virtual_file_systems.html) that allows you to clip rasters on Google Cloud Storage. 

We'll run through an example now to use a `geojson` file to download canopy height data from the perimeter of the CZU fire in the Santa Cruz Mountains. This will use `gdal`'s python bindings, but we'll also show what the command line version would look like.

In [ ]:
# this vector is in cfo-api/demos
vector = os.path.join(os.getcwd(), "czu-perimeter.geojson")

# get the 2020 10m canopy height asset id
czu_search = forest.search(geography='California', metric='CanopyHeight', year=2020, resolution=10)
czu = czu_search[0]

# and get the gdal file path reference
input_file = forest.fetch(czu, gdal=True)
print(f"We'll download from: {input_file}")

In [ ]:
# set the output raster file
output_file = os.path.join(output_directory, f"{czu}.tif")

# set the gdalwarp options
options = gdal.WarpOptions(
    creationOptions = ["COMPRESS=DEFLATE", "TILED=YES", "BIGTIFF=YES", "NUM_THREADS=ALL_CPUS"],
    cutlineDSName = vector,
    cropToCutline = True
)

# and run the command
warp = gdal.Warp(output_file, input_file, options=options)
warp.FlushCache()

The above is akin to running something like this at the command line:

```
gdalwarp /vsigs/cfo-public/vegetation/California-Vegetation-CanopyHeight-2020-Summer-00010m.tif /home/cba/cfo/California-Vegetation-CanopyHeight-2020-Summer-00010m.tif -cutline /home/cba/src/cfo-api/demos/czu-perimeter.geojson -crop_to_cutline
```

In [ ]:
# read and mask the data
source = rasterio.open(output_file)
height = source.read(1).astype(float)
height[height == source.nodata] = np.nan

# get the range to show
vmin = np.nanpercentile(height, 5)
vmax = np.nanpercentile(height, 95)

# and plot
plt.plot(figsize=(15,15), dpi=100)
height_map = plt.imshow(height, vmin=vmin, vmax=vmax, cmap=plt.cm.cividis)
plt.title("Canopy height inside the\nburn perimeter of the CZU fire")
colorbar = plt.colorbar(height_map)
colorbar.set_label("Canopy Height (m)")
plt.tight_layout()

# Web mapping

In [ ]:
forest.fetch("MendocinoCounty-Vegetation-CanopyHeight-2020-Fall-00010m", wms=True)

WMS URLs don't always easily plug and play with different rendering services, but they should work with a little nudging. Here's how to use the above URL to visualize these data with `ipyleaflet`.

In [ ]:
wms = WMSLayer(
    url='https://maps.salo.ai/geoserver/cfo/wms?p0=0.0&p2=1.44&p25=18.0&p30=21.599999999999998&p50=36.0&p60=43.199999999999996&p75=54.0&p90=64.8&p98=70.56&p100=72.0',
    layers="cfo:MendocinoCounty-Vegetation-CanopyHeight-2020-Summer-00010m",
    name="Mendocino Canopy Height",
    styles="vegetation",
    format="image/png8",
    transparent=True,
    attribution="Forest Observatory © <a href=https://salo.ai'>Salo Sciences</a>",
)
m = Map(basemap=basemaps.Stamen.Terrain, center=(39.39,-123.33), zoom=10)
m.add_layer(wms)
control = LayersControl(position='topright')
m.add_control(control)
m